# 제주 관광객 소비패턴 변화 분석(BC카드)
## 1. Definition
### Overview
2014년~2016년 제주도 내 내국인 관광객의 카드 소비 패턴 분석을 통해서 어떠한 상권에서 어떠한 소비를 많이 하는지에 대해 분석하고, 이를 통해 데이터가 주어지면 어떠한 상권에서 어떠한 업종이 유망할 지에 대한 모델을 만들고자 한다.
### Metric
## 2. Data Exploration
### import library

In [12]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from glob import glob
import os, random, time, gc, warnings

from tqdm import tqdm_notebook

import lightgbm as lgbm
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

# from catboost import CatBoostRegressor
from sklearn.feature_selection import RFECV


from sklearn.cluster import KMeans

from datetime import datetime

from math import sqrt

import folium
from folium import Marker, Icon, CircleMarker
from folium.plugins import MarkerCluster

from pdpbox import pdp, info_plots

warnings.filterwarnings('ignore')

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

%matplotlib inline

In [13]:
os.listdir('./input/')

['domestic_consumption_data.csv',
 'market_consumption_data.csv',
 'reference_data',
 'supplementary_data']

### Load Dataset

In [87]:
data = pd.read_csv('./input/domestic_consumption_data.csv',parse_dates=['기준년월','데이터기준일자'],encoding='ANSI')
data.head()

,기준년월,관광객 유형,제주 대분류,제주 중분류,업종명,성별,연령대별,카드이용금액,카드이용건수,건당이용금액,데이터기준일자
0,2014-09-01,내국인 관광객,서귀포시,대륜동,농축수산품,여,50대,14434000,67,215433,2017-02-16
1,2014-09-01,내국인 관광객,서귀포시,대륜동,농축수산품,남,50대,15119000,72,209986,2017-02-16
2,2014-09-01,내국인 관광객,서귀포시,대륜동,농축수산품,여,40대,7609500,41,185598,2017-02-16
3,2014-09-01,내국인 관광객,서귀포시,남원읍,농축수산품,남,50대,7092500,40,177313,2017-02-16
4,2014-09-01,내국인 관광객,서귀포시,대륜동,농축수산품,남,40대,9098500,59,154212,2017-02-16


In [15]:
data.columns

Index(['기준년월', '관광객 유형', '제주 대분류', '제주 중분류', '업종명', '성별', '연령대별', ' 카드이용금액 ',
       ' 카드이용건수 ', '건당이용금액', '데이터기준일자'],
      dtype='object')

### Columns
- 기준년월 : 해당 데이터가 발생한 날짜
- 관광객 유형 : 모두 내국인 관광객으로 동일
- 제주 대분류 : 제주시, 서귀포시
- 제주 중분류 : 읍면동
- 업종명 : 업종의 종류를 나타낸다.
- 성별 : 남/여
- 연령대별 : 연령대
- 카드이용금액
- 카드이용건수
- 건당이용금액 : 카드이용금액 / 카드이용건수
- 데이터기준일자 : database가 만들어진 날짜

'데이터 기준일자'은 2017-02-16으로 모두 동일하고,  '관광객 유형'은 모두 내국인 관광객으로 동일하므로 삭제한다.

In [89]:
data.drop(['관광객 유형','데이터기준일자'],axis=1,inplace=True)
data.columns

Index(['기준년월', '제주 대분류', '제주 중분류', '업종명', '성별', '연령대별', ' 카드이용금액 ', ' 카드이용건수 ',
       '건당이용금액'],
      dtype='object')

중복되는 데이터는 없다.

In [91]:
display(data.shape)
display(data.duplicated().sum())

(13146, 9)

0

### Missing Value
결측치는 없다.

In [33]:
# 한글 글꼴 설정
import matplotlib.font_manager as fm

font_path = r'./NanumFontSetup_TTF_BARUNGOTHIC/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)

In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13146 entries, 0 to 13145
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   기준년월      13146 non-null  datetime64[ns]
 1   제주 대분류    13146 non-null  object        
 2   제주 중분류    13146 non-null  object        
 3   업종명       13146 non-null  object        
 4   성별        13146 non-null  object        
 5   연령대별      13146 non-null  object        
 6    카드이용금액   13146 non-null  int64         
 7    카드이용건수   13146 non-null  int64         
 8   건당이용금액    13146 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 924.5+ KB


#### 제주 대분류, 제주 중분류

In [93]:
data.columns

Index(['기준년월', '제주 대분류', '제주 중분류', '업종명', '성별', '연령대별', ' 카드이용금액 ', ' 카드이용건수 ',
       '건당이용금액'],
      dtype='object')

In [94]:
city_in_jejudo = data['제주 대분류'].unique()
town_in_jejudo = data['제주 중분류'].unique()
display(print('도시의 갯수 : {}'.format(len(city_in_jejudo))))
display(print(city_in_jejudo))
display(print('동네의 갯수 : {}'.format(len(town_in_jejudo))))
display(print(town_in_jejudo))

도시의 갯수 : 2


None

['서귀포시' '제주시']


None

동네의 갯수 : 10


None

['대륜동' '남원읍' '연동' '예래동' '이도2동' '삼도2동' '용담2동' '노형동' '애월읍' '성산읍']


None

#### 업종명
중간에 불필요한 공백이 있는 data는 공백을 없애주도록 한다.

In [95]:
upjong = data['업종명'].unique()
print(f'업종의 갯수 : {len(upjong)}\n',upjong)

업종의 갯수 : 14
 ['농축수산품' '스포츠레져용품' '골프 용품' '악세 사리' '의원' '안경' '정장(여성)' '기념품 점' '기타음료식품'
 '슈퍼 마켓' '신   발' '약국' '스넥' '귀 금 속']


In [96]:
# 불필요한 공백 제거
data['업종명'] = [uj.replace(" ","") for uj in data['업종명']]
data['업종명'].unique()

array(['농축수산품', '스포츠레져용품', '골프용품', '악세사리', '의원', '안경', '정장(여성)', '기념품점',
       '기타음료식품', '슈퍼마켓', '신발', '약국', '스넥', '귀금속'], dtype=object)

#### 성별

In [97]:
sex = data['성별'].unique()
print(sex)

['여' '남']


#### 연령대별

In [98]:
age_grp = data['연령대별'].unique()
print(age_grp)

['50대' '40대' '30대' '20대']


#### 카드이용금액

In [99]:
data.columns

Index(['기준년월', '제주 대분류', '제주 중분류', '업종명', '성별', '연령대별', ' 카드이용금액 ', ' 카드이용건수 ',
       '건당이용금액'],
      dtype='object')